In [29]:
# import dependencies
import json
import requests
import pandas as pd
from config import app_token

In [43]:
# read in csv file for crimes from one year prior to present
pastyear_data = pd.read_csv("Data/crimes_pastyear_topresent.csv", encoding="utf-8")

pastyear_data.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,JE145602,12/22/2020 08:59:00 PM,004XX W 58TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,N,N,711,20.0,11,NaN,NaN,NaN,NaN,NaN
1,JE173033,02/01/2021 09:30:00 PM,001XX E DELAWARE PL,0890,THEFT,FROM BUILDING,RESIDENCE,N,N,1833,2.0,06,NaN,NaN,NaN,NaN,NaN
2,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)"
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)"
4,JD381795,09/13/2020 12:00:00 PM,015XX N KINGSBURY ST,0810,THEFT,OVER $500,GROCERY FOOD STORE,Y,N,1822,27.0,06,1169505.0,1910252.0,41.909240,-87.652748,"(41.909240284, -87.65274758)"


In [44]:
# drop columns that will not be necessary for analysis
clean_pastyear = pastyear_data.drop(columns=['BLOCK', ' IUCR', ' SECONDARY DESCRIPTION',' LOCATION DESCRIPTION','ARREST',
                                             'DOMESTIC','FBI CD','X COORDINATE','Y COORDINATE','LOCATION'])

clean_pastyear

,CASE#,DATE OF OCCURRENCE,PRIMARY DESCRIPTION,BEAT,WARD,LATITUDE,LONGITUDE
0,JE145602,12/22/2020 08:59:00 PM,DECEPTIVE PRACTICE,711,20.0,NaN,NaN
1,JE173033,02/01/2021 09:30:00 PM,THEFT,1833,2.0,NaN,NaN
2,JD220983,04/28/2020 04:00:00 PM,BURGLARY,822,14.0,41.796167,-87.709519
3,JD390623,09/30/2020 12:00:00 PM,THEFT,1814,43.0,41.915454,-87.638727
4,JD381795,09/13/2020 12:00:00 PM,THEFT,1822,27.0,41.909240,-87.652748
...,...,...,...,...,...,...,...
200248,JE108866,01/04/2021 05:45:00 PM,THEFT,523,34.0,NaN,NaN
200249,JD389381,10/02/2020 05:39:00 PM,INTIMIDATION,815,23.0,41.796261,-87.731827
200250,JD362186,09/09/2020 10:41:00 PM,ROBBERY,2432,40.0,41.998104,-87.668264
200251,JD402493,10/17/2020 03:15:00 PM,BATTERY,935,3.0,41.797472,-87.633075


In [45]:
# rename columns for readability
clean_pastyear = clean_pastyear.rename(columns={'CASE#':'CASE #','DATE  OF OCCURRENCE':'DATE OF OCCURRENCE',' PRIMARY DESCRIPTION':'PRIMARY DESCRIPTION','WARD':'DISTRICT #'})

clean_pastyear

,CASE #,DATE OF OCCURRENCE,PRIMARY DESCRIPTION,BEAT,DISTRICT #,LATITUDE,LONGITUDE
0,JE145602,12/22/2020 08:59:00 PM,DECEPTIVE PRACTICE,711,20.0,NaN,NaN
1,JE173033,02/01/2021 09:30:00 PM,THEFT,1833,2.0,NaN,NaN
2,JD220983,04/28/2020 04:00:00 PM,BURGLARY,822,14.0,41.796167,-87.709519
3,JD390623,09/30/2020 12:00:00 PM,THEFT,1814,43.0,41.915454,-87.638727
4,JD381795,09/13/2020 12:00:00 PM,THEFT,1822,27.0,41.909240,-87.652748
...,...,...,...,...,...,...,...
200248,JE108866,01/04/2021 05:45:00 PM,THEFT,523,34.0,NaN,NaN
200249,JD389381,10/02/2020 05:39:00 PM,INTIMIDATION,815,23.0,41.796261,-87.731827
200250,JD362186,09/09/2020 10:41:00 PM,ROBBERY,2432,40.0,41.998104,-87.668264
200251,JD402493,10/17/2020 03:15:00 PM,BATTERY,935,3.0,41.797472,-87.633075


In [48]:
# filter the dataframe to display only homicide incidents
homicide_pastyear = clean_pastyear.loc[clean_pastyear['PRIMARY DESCRIPTION'] == 'HOMICIDE']

homicide_pastyear = homicide_pastyear.reset_index()

del homicide_pastyear['index']

homicide_pastyear

,CASE #,DATE OF OCCURRENCE,PRIMARY DESCRIPTION,BEAT,DISTRICT #,LATITUDE,LONGITUDE
0,JD424989,11/08/2020 01:22:00 PM,HOMICIDE,831,17.0,41.761848,-87.702741
1,JD448552,12/03/2020 02:45:00 PM,HOMICIDE,2212,19.0,41.679104,-87.681114
2,JD240881,05/23/2020 11:53:00 PM,HOMICIDE,1422,26.0,41.910030,-87.713732
3,JD455780,12/11/2020 04:50:00 AM,HOMICIDE,231,3.0,41.795683,-87.619251
4,JD425287,11/08/2020 06:39:00 PM,HOMICIDE,612,17.0,41.751583,-87.650128
...,...,...,...,...,...,...,...
808,JE102438,01/03/2021 08:09:00 PM,HOMICIDE,332,5.0,41.771062,-87.586271
809,JE103492,01/04/2021 10:40:00 PM,HOMICIDE,711,3.0,41.791735,-87.632921
810,JC461838,11/09/2020 12:45:00 AM,HOMICIDE,414,8.0,41.758120,-87.574905
811,JE103451,01/04/2021 08:54:00 PM,HOMICIDE,1133,24.0,41.870831,-87.716247


In [49]:
# read in csv file for locations of police stations in Chicago
police_stations = pd.read_csv("Data/police_stations.csv")

police_stations

,DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,NaN,NaN,NaN,1177731.401,1881697.404,41.830702,-87.623395,"(41.8307016873, -87.6233953459)"
1,18,Near North,1160 N Larrabee St,Chicago,IL,60610,http://home.chicagopolice.org/community/distri...,312-742-5870,312-742-5771,312-742-5773,1172080.029,1908086.527,41.903242,-87.643352,"(41.9032416531, -87.6433521393)"
2,19,Town Hall,850 W Addison St,Chicago,IL,60613,http://home.chicagopolice.org/community/distri...,312-744-8320,312-744-4481,312-744-8011,1169730.744,1924160.317,41.947400,-87.651512,"(41.9474004564, -87.651512018)"
3,20,Lincoln,5400 N Lincoln Ave,Chicago,IL,60625,http://home.chicagopolice.org/community/distri...,312-742-8714,312-742-8803,312-742-8841,1158399.146,1935788.826,41.979550,-87.692845,"(41.9795495131, -87.6928445094)"
4,22,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643,http://home.chicagopolice.org/community/distri...,312-745-0710,312-745-0814,312-745-0569,1165825.476,1830851.333,41.691435,-87.668520,"(41.6914347795, -87.6685203937)"
5,24,Rogers Park,6464 N Clark St,Chicago,IL,60626,http://home.chicagopolice.org/community/distri...,312-744-5907,312-744-6928,312-744-7603,1164193.588,1943199.401,41.999763,-87.671324,"(41.9997634842, -87.6713242922)"
6,25,Grand Central,5555 W Grand Ave,Chicago,IL,60639,http://home.chicagopolice.org/community/distri...,312-746-8605,312-746-4353,312-746-8383,1138770.871,1913442.439,41.918609,-87.765574,"(41.9186088912, -87.765574479)"
7,1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/distri...,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.858373,-87.627356,"(41.8583725929, -87.627356171)"
8,2,Wentworth,5101 S Wentworth Ave,Chicago,IL,60609,http://home.chicagopolice.org/community/distri...,312-747-8366,312-747-5396,312-747-6656,1175864.837,1871153.753,41.801811,-87.630560,"(41.8018110912, -87.6305601801)"
9,3,Grand Crossing,7040 S Cottage Grove Ave,Chicago,IL,60637,http://home.chicagopolice.org/community/distri...,312-747-8201,312-747-5479,312-747-9168,1182739.183,1858317.732,41.766431,-87.605748,"(41.7664308925, -87.6057478606)"


In [51]:
# drop columns that will not be necessary for analysis
clean_police_stations = police_stations.drop(columns=['WEBSITE','PHONE','FAX','TTY','X COORDINATE','Y COORDINATE','LOCATION'])

clean_police_stations

,DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,LATITUDE,LONGITUDE
0,Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,41.830702,-87.623395
1,18,Near North,1160 N Larrabee St,Chicago,IL,60610,41.903242,-87.643352
2,19,Town Hall,850 W Addison St,Chicago,IL,60613,41.947400,-87.651512
3,20,Lincoln,5400 N Lincoln Ave,Chicago,IL,60625,41.979550,-87.692845
4,22,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643,41.691435,-87.668520
5,24,Rogers Park,6464 N Clark St,Chicago,IL,60626,41.999763,-87.671324
6,25,Grand Central,5555 W Grand Ave,Chicago,IL,60639,41.918609,-87.765574
7,1,Central,1718 S State St,Chicago,IL,60616,41.858373,-87.627356
8,2,Wentworth,5101 S Wentworth Ave,Chicago,IL,60609,41.801811,-87.630560
9,3,Grand Crossing,7040 S Cottage Grove Ave,Chicago,IL,60637,41.766431,-87.605748
